### 1.data load

In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

#  data load
df2019 = pd.read_csv("../data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

In [201]:
df2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83940 entries, 0 to 83939
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종목코드    83940 non-null  object 
 1   회사명     83940 non-null  object 
 2   시장구분    83940 non-null  object 
 3   업종      83940 non-null  int64  
 4   업종명     83940 non-null  object 
 5   결산월     83940 non-null  int64  
 6   결산기준일   83940 non-null  object 
 7   보고서종류   83940 non-null  object 
 8   통화      83940 non-null  object 
 9   항목코드    83940 non-null  object 
 10  항목명     83940 non-null  object 
 11  당기      75426 non-null  float32
dtypes: float32(1), int64(2), object(9)
memory usage: 7.4+ MB


### 2. 불필요한 변수 제거

In [169]:
# 필요 없는 변수 제거
def delete_col(data):
#     data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("전기", axis = 1, inplace = True)
    data.drop("전전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df2019 = delete_col(df2019)

### 3. 연속형변수(당기, 전기, 전전기)  dtype 변경

In [170]:
def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
#     data["전기"] = data["전기"].str.replace(",", "")
#     data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    return data

df2019 = str_to_float(df2019)

### 4. 항목명 변수 처리

In [171]:
def duplication_check(data):
    # 항목코드 중복 확인
    entity_코드 = []
    entity_명 = []
    without_entity_코드 = []
    without_entity_명 = []

    # 각 리스트에 코드 및 코드명 append(소문자 처리)
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            entity_코드.append(data["항목코드"][idx].lower())
            entity_명.append(data["항목명"][idx].lower())
        else:
            without_entity_코드.append(data["항목코드"][idx].lower())
            without_entity_명.append(data["항목명"][idx].lower())

    # !entity 코드 딕셔너리
    dic_without_entity = {re.findall("_\w{1,}", without_entity_코드[idx])[0].lower():[] for idx in range(len(without_entity_코드))}

    for i, j in zip(without_entity_코드, without_entity_명):
        dic_without_entity[re.findall("_\w{1,}", i)[0].lower()].append(j)

    check_without_entity = {i:len(set(j)) for i, j in dic_without_entity.items()}
    
    # entity 코드 딕셔너리
    dic_entity = {entity_코드[idx]:[] for idx in range(len(entity_코드))}
    
    for i, j in zip(entity_코드, entity_명):
        dic_entity[i].append(j)
        
    check_entity = {i:len(set(j)) for i, j in dic_entity.items()}
    
    return entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity

entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity = duplication_check(df2019)

### 5. 공시된 항목명, 항목코드 가져오기(금융감독원 재무제표 양식)

In [172]:
def extract_element_id(data):
    element_idx_lst_entity = []
    element_idx_lst_without_entity = []
    
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            element_idx_lst_entity.append(data["항목코드"][idx])
        else:
            element_idx_lst_without_entity.append(data["항목코드"][idx])
            
    entity = sorted(set(element_idx_lst_entity), reverse = True)
    without_entity = sorted(set(element_idx_lst_without_entity), reverse = True)
    
    return entity, without_entity

a, b = extract_element_id(df2019)
pd.Series(b).to_csv("../2019항목명.csv", encoding = "euc-kr", index = False)

elementid = pd.read_excel("../data/재무제표양식.xlsx", encoding = "utf-8", sheet_name = "BS1")
# '한글 Label', 'Element ID'
df_element = elementid[['한글 Label', 'Element ID']]
df_element = df_element[df_element["Element ID"].notnull()]

df_element.columns = df_element.columns.str.replace(" ", "_")
lst_element = df_element["Element_ID"].unique()

re_lst_element = []
for i in lst_element:
    a = re.findall("_\w{1,}", i)[0].lower()
    re_lst_element.append(a)
 
dic_element = {i:[] for i in re_lst_element}
for i, j in zip(re_lst_element, df_element["한글_Label"]):
    dic_element[i].append(j)

### 5-1. 항목명 entity 값 처리(ver_1)

In [173]:
# 항목명 전치리

# 1) _가진 항목명 추출
lst_ = []
for i in range(len(df2019)):
    if "_" in df2019.loc[i, "항목명"]:
        lst_.append(i)
        
def preprocessing(x):
    a = x.replace(" ", "")
    a = a.replace("_", "")
    a = a.replace("[", "")
    a = a.replace("]", "")
    a = a.replace("(", "")
    a = a.replace(")", "")
    a = a.replace(".", "")
    a = re.sub("[ⅠⅡⅢⅣⅤIII]", "", a)
    a = re.sub("[1-9]", "", a)
    
    
    return a

df2019["항목명"] = df2019["항목명"].agg(preprocessing)

In [174]:
lst_idx = []
for i in range(len(df2019)):
    if "총계" in df2019["항목명"][i] and "entity" in df2019["항목코드"][i]:
        lst_idx.append(i)

In [ ]:
for i in lst_idx:
    if df2019["항목명"][i] == '부채총계':
        df2019["항목코드"][i] = "ifrs-full_Liabilities"
        
    elif df2019["항목명"][i] == '자본과부채의총계':
        df2019["항목코드"][i] = "ifrs-full_EquityAndLiabilities"
        
    elif df2019["항목명"][i] == '자산총계':
        df2019["항목코드"][i] = "ifrs-full_Assets"
        
    else:
        df2019["항목코드"][i] = "ifrs-full_Equity"

### 6. entity 제외 항목명 변수 전처리

In [176]:
for idx in range(len(df2019)):
    if "entity" not in df2019["항목코드"][idx]:
        df2019["항목명"][idx] = dic_element[re.findall("_\w{1,}", df2019["항목코드"][idx])[0].lower()][0]

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### 6-1. entity 변수 전처리

In [ ]:
#  항목명 여러개 확인

dic_please = {i:[]  for i in df2019["항목명"].unique()}

for i in range(len(df2019)):
    if "entity" not in df2019.loc[i, "항목코드"]:
        dic_please[df2019.loc[i, "항목명"]].append(df2019.loc[i, "항목코드"])
        
for i,j in dic_please.items():
    dic_please[i] = set(j)       
    
dic_final = {}
cnt = 0
for i, j in dic_please.items():
    if len(j) == 2:
        j = list(j)
        if re.findall("_\w{1,}", j[0])[0] != re.findall("_\w{1,}", j[1])[0]:
            dic_final[i] = j
            cnt += 1
    elif len(j) > 2:
        j = list(j)
        dic_final[i] = j
        cnt += 1
        
print(cnt)

dic_final.keys()
# 제거 : 감가상각누계액, 정부보조금
# 합계 : 대손충당금, 현재가치할인차금
# 보류 : 평가충당금, 손상차손누계액
df2019.drop((df2019[df2019["항목명"] == "감가상각누계액") | (df2019[df2019["항목명"] == "정부보조금"]).index, inplace = True).reset_index(drop = True, inplace = True)

In [196]:
# entity변수를 가진 index 추출
idx_entity = []

for i in range(len(df2019)):
    if "entity" in df2019.loc[i, "항목코드"]:
        idx_entity.append(i)

In [197]:
elementid = pd.read_excel("../data/재무제표양식.xlsx", encoding = "utf-8", sheet_name = "BS1")
# '한글 Label', 'Element ID'
df_element = elementid[['한글 Label', 'Element ID']]
df_element = df_element[df_element["Element ID"].notnull()]

df_element.columns = df_element.columns.str.replace(" ", "_")
lst_element = df_element["Element_ID"].unique()

re_lst_element = []
for i in lst_element:
    a = re.findall("_\w{1,}", i)[0].lower()
    re_lst_element.append(a)
 
dic_elementid = {(((i.replace("_", "")).replace(" ", "")).replace("[", "")).replace("]", ""):j 
                 for i, j in zip(df_element["한글_Label"], df_element["Element_ID"])}


# entity 포함 항목코드 항목코드 전처리
no_preprocessing_entity_idx = []
for idx in idx_entity:
    try:
        df2019.loc[idx, "항목코드"] = dic_elementid[df2019.loc[idx, "항목명"]]

    except Exception as e:
        no_preprocessing_entity_idx.append(idx)
        
# 한번더 전처리
for idx in range(len(df2019)):
    if "entity" not in df2019["항목코드"][idx]:
        df2019["항목명"][idx] = dic_element[re.findall("_\w{1,}", df2019["항목코드"][idx])[0].lower()][0]

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [200]:
df2019.iloc[no_preprocessing_entity_idx,]["항목명"].value_counts()

기타수취채권             134
무형자산               130
당기손익-공정가치측정금융자산    123
기타지급채무              96
매각예정비유동자산           90
                  ... 
유동성차입금및사채            1
유동장기리스부채             1
지속관여자산관련부채           1
매각예정비유동자산관련부채        1
유동화부채                1
Name: 항목명, Length: 957, dtype: int64

In [207]:
df2019[df2019["항목명"] == "유동부채"]["항목코드"].unique()

array(['ifrs_CurrentLiabilities', 'ifrs-full_CurrentLiabilities'],
      dtype=object)

### 7. 정제된 데이터 프레임 만들기 

In [65]:
# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df2019["회사명"].unique()
element_value = []
for i in dic_element.values():
    element_value.append(i[0])
    
# columns
col = list(df2019.columns[0:8]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df2019[df2019["회사명"] == i]["결산기준일"].unique())
    day.append(len(df2019[df2019["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
last_df = pd.DataFrame(bon, columns = col)

### 8. 데이터 입력

In [67]:
dic_standard = {i:list(df2019[df2019["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
        
cnt = 0

for idx in range(len(last_company)):

    a = df2019[(df2019["회사명"] == last_company[idx]) & (df2019["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(8):
        last_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = dic_element[re.findall("_\w{1,}", a["항목코드"][idx3])[0].lower()][0]
            last_df.loc[cnt, loc] = a["당기"][idx3]
    cnt += 1

### 9. 2018데이터 입력

In [ ]:
# 2018년 변수 처리
df = pd.read_csv("./data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
    data["전기"] = data["전기"].str.replace(",", "")
    data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    data["전기"] = data["전기"].astype(np.float32)
    data["전전기"] = data["전전기"].astype(np.float32)
    return data

df = str_to_float(df)

# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("당기", axis = 1, inplace = True)
    data.drop("전전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df = delete_col(df)


# 항목명 변수 전처리
lst_idx = []
for i in range(len(df)):
    if "총계" in df["항목명"][i] and "entity" in df["항목코드"][i]:
        lst_idx.append(i)


for i in lst_idx:
    if df["항목명"][i] == '부채총계':
        df["항목코드"][i] = "ifrs-full_Liabilities"
        
    elif df["항목명"][i] == '자본과 부채의 총계':
        df["항목코드"][i] = "ifrs-full_EquityAndLiabilities"
        
    elif df["항목명"][i] == '자산총계':
        df["항목코드"][i] = "ifrs-full_Assets"
        
    else:
        df["항목코드"][i] = "ifrs-full_Equity"
        
        
for idx in range(len(df)):
    if "entity" not in df["항목코드"][idx]:
        df["항목명"][idx] = dic_element[re.findall("_\w{1,}", df["항목코드"][idx])[0].lower()][0]




# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df["회사명"].unique()
element_value = []
for i in dic_element.values():
    element_value.append(i[0])
    
# columns
col = list(df.columns[0:8]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df[df["회사명"] == i]["결산기준일"].unique())
    day.append(len(df[df["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
add_df = pd.DataFrame(bon, columns = col)


# 정제된 데이터프레임 생성
dic_standard = {i:list(df[df["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
        
cnt = 0

for idx in range(len(last_company)):

    a = df[(df["회사명"] == last_company[idx]) & (df["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(8):
        add_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = dic_element[re.findall("_\w{1,}", a["항목코드"][idx3])[0].lower()][0]
            add_df.loc[cnt, loc] = a["전기"][idx3]
    cnt += 1

add_df["결산기준일"] = add_df["결산기준일"].str.replace("2019", "2018")

last_df = pd.concat([last_df, add_df]).reset_index(drop = True)

### 10. 2017 데이터 입력

In [ ]:
# 2017년 변수 처리
df = pd.read_csv("./data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
    data["전기"] = data["전기"].str.replace(",", "")
    data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    data["전기"] = data["전기"].astype(np.float32)
    data["전전기"] = data["전전기"].astype(np.float32)
    return data

df = str_to_float(df)

# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("당기", axis = 1, inplace = True)
    data.drop("전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df = delete_col(df)

# 항목명 변수 전처리
lst_idx = []
for i in range(len(df)):
    if "총계" in df["항목명"][i] and "entity" in df["항목코드"][i]:
        lst_idx.append(i)


for i in lst_idx:
    if df["항목명"][i] == '부채총계':
        df["항목코드"][i] = "ifrs-full_Liabilities"
        
    elif df["항목명"][i] == '자본과 부채의 총계':
        df["항목코드"][i] = "ifrs-full_EquityAndLiabilities"
        
    elif df["항목명"][i] == '자산총계':
        df["항목코드"][i] = "ifrs-full_Assets"
        
    else:
        df["항목코드"][i] = "ifrs-full_Equity"
        
        
for idx in range(len(df)):
    if "entity" not in df["항목코드"][idx]:
        df["항목명"][idx] = dic_element[re.findall("_\w{1,}", df["항목코드"][idx])[0].lower()][0]

# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df2019["회사명"].unique()
element_value = []
for i in dic_element.values():
    element_value.append(i[0])
    
# columns
col = list(df.columns[0:8]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df[df["회사명"] == i]["결산기준일"].unique())
    day.append(len(df[df["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
add_df = pd.DataFrame(bon, columns = col)


dic_standard = {i:list(df[df["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
        
cnt = 0

for idx in range(len(last_company)):

    a = df[(df["회사명"] == last_company[idx]) & (df["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(8):
        add_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = dic_element[re.findall("_\w{1,}", a["항목코드"][idx3])[0].lower()][0]
            add_df.loc[cnt, loc] = a["전전기"][idx3]
    cnt += 1

add_df["결산기준일"] = add_df["결산기준일"].str.replace("2019", "2017")

last_df = pd.concat([last_df, add_df]).reset_index(drop = True)

In [76]:
last_df.to_csv("./data/2017_2019_without_entity.csv", encoding = "euc-kr", index = False)